In [385]:
import pandas as pd

national_results = pd.read_csv('/Users/danielsagher/Dropbox/Documents/General Assembly/Class Projects/capstone/csv/national_results.csv')
state_results = pd.read_csv('/Users/danielsagher/Dropbox/Documents/General Assembly/Class Projects/capstone/csv/state_results.csv')
national_series_dimension = pd.read_csv('/Users/danielsagher/Dropbox/Documents/General Assembly/Class Projects/capstone/csv/national_series_dimension.csv')
state_series_dimension = pd.read_csv('/Users/danielsagher/Dropbox/Documents/General Assembly/Class Projects/capstone/csv/state_series_dimension.csv')
survey_table = pd.read_csv('/Users/danielsagher/Dropbox/Documents/General Assembly/Class Projects/capstone/csv/survey_table.csv')



In [386]:
national_results = national_results.drop(columns='Unnamed: 0')
national_series_dimension = national_series_dimension.drop(columns='Unnamed: 0')
state_results = state_results.drop(columns='Unnamed: 0')
state_series_dimension = state_series_dimension.drop(columns='Unnamed: 0')

In [387]:
pd.merge(national_results[(national_results.duplicated() == True)], national_series_dimension, how='left')\
        .series.unique()

array(['Civilian Labor Force Participation Rate ', 'Employment Level ',
       'Employment Population Ratio ', 'Employment - Population Ratio ',
       'Employed, Usually Work Full Time ',
       'Employed, Usually Work Part Time '], dtype=object)

In [400]:
national_results = national_results.drop_duplicates()
national_results.duplicated().sum()

0

In [389]:
national_results.dtypes

seriesID       object
year            int64
period         object
period_name    object
value          object
footnotes      object
dtype: object

In [390]:
national_results['value'] = national_results['value'].replace('-', 0)
national_results['value'] = national_results['value'].astype('float')

national_results.dtypes

seriesID        object
year             int64
period          object
period_name     object
value          float64
footnotes       object
dtype: object

In [391]:

def space_remover(df):

        for col in df.columns:

                df[col] = df[col].apply(lambda x: x.strip() if isinstance(x, str) else x)
                
        return df


national_results = space_remover(national_results)
national_series_dimension = space_remover(national_series_dimension)
state_results = space_remover(state_results)
state_series_dimension = space_remover(state_series_dimension)
survey_table = space_remover(survey_table)
                

In [392]:
national_results.footnotes = [i.replace('[{}]', 'NaN') for i in national_results.footnotes]
state_results.footnotes = [i.replace('[{}]', 'NaN') for i in state_results.footnotes]


In [393]:
national_series_dimension['is_adjusted'] = \
    national_series_dimension['series'].str.lower().apply(lambda x: False if 'not seasonally adjusted' in x else True)

state_series_dimension['is_adjusted'] = \
   state_series_dimension['series'].str.lower().apply(lambda x: False if 'not seasonally adjusted' in x else True)


In [394]:
state_series_dimension.series[(state_series_dimension.is_adjusted == False)] 
national_series_dimension.series[(national_series_dimension.is_adjusted == False)]

119    Total Nonfarm Employment, Not Seasonally Adjusted
135    Private Education and Health Services Average ...
137    Total Private Employment, Not Seasonally Adjusted
172    Job openings level, Total nonfarm, not seasona...
173    Job openings rate, Total nonfarm, not seasonal...
174    Hires level, Total nonfarm, not seasonally adj...
175    Total separations rate, Total nonfarm, not sea...
176    Quits rate, Total nonfarm, not seasonally adju...
177    Layoffs and discharges rate, Total nonfarm, no...
178    Job openings level, Construction, not seasonal...
179    Job openings level, Manufacturing, not seasona...
Name: series, dtype: object

In [395]:
# national_series_dimension.series = national_series_dimension['series'].str.lower().replace('seasonally adjusted', '').replace('not', '')
# state_series_dimension['series'] = state_series_dimension['series'].str.replace('Seasonally adjusted', '').replace('Not', '')

def adjusted_remover(series):
    term1 = ', seasonally adjusted'
    term2 = ', not seasonally adjusted'
    
    def remove_terms(text):
        text_lower = text.lower()
        index1 = text_lower.find(term1)
        index2 = text_lower.find(term2)
        if index1 != -1:
            return text[:index1]
        elif index2 != -1:
            return text[:index2]
        return text
    
    # Apply the removal function to the 'series' column
    series['series'] = series['series'].apply(remove_terms)
    return series

# adjusted_remover(national_series_dimension)
national_series_dimension = adjusted_remover(national_series_dimension)
state_series_dimension = adjusted_remover(state_series_dimension)



In [396]:
national_series_dimension[(national_series_dimension['series'].str.contains('seasonally adjusted') == True)].sum(),\
state_series_dimension[(state_series_dimension['series'].str.contains('seasonally adjusted') == True)].sum()

(series         0
 seriesID       0
 survey         0
 is_adjusted    0
 dtype: object,
 series         0
 seriesID       0
 state          0
 is_adjusted    0
 survey         0
 dtype: object)

In [399]:
national_series_dimension.to_csv('/Users/danielsagher/Dropbox/Documents/General Assembly/Class Projects/capstone/csv/national_seriesID_list.csv', index=False)
state_series_dimension.to_csv('/Users/danielsagher/Dropbox/Documents/General Assembly/Class Projects/capstone/csv/state_series_dimension.csv', index=False)